In [1]:
from common import *

### Generate datareader and loader

In [3]:
subset = 'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z-STEP_ROT_0.5-SINU_PITCH_M15_P273-SINU_ROLL_M15_P273-TR_S1_TS_S5678'
datareader = DataReaderTotal(subset=subset)

[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d_same_z-steprot_0.5-sinu_pitch_m15_p273-sinu_roll_m15_p273...


  1%|▏         | 12/836 [00:00<00:06, 119.72it/s]

100%|██████████| 836/836 [00:10<00:00, 78.06it/s] 


[overwrite: False] ==> Loading H36M cam_3d_canonical_same_z-steprot_0.5-sinu_pitch_m15_p273-sinu_roll_m15_p273...


100%|██████████| 836/836 [00:06<00:00, 128.32it/s]


[overwrite: False] ==> Loading H36M img_2d_canonical_same_z-steprot_0.5-sinu_pitch_m15_p273-sinu_roll_m15_p273...


100%|██████████| 836/836 [00:00<00:00, 1020.06it/s]


In [ ]:
checkpoint = 'MB_train_h36m_gt'
args, opts = get_opt_args_from_model_name(checkpoint)
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)

configs/pose3d/MB_train_h36m_gt.yaml
True
Loading dataset...
H36M-GT
==> Loading H36M source_list... overwrite: False
==> Loading H36M cam_param... overwrite: False
==> Loading H36M cam_3d... overwrite: False
==> Loading H36M img_3d... overwrite: False
==> Loading H36M scale_factor... overwrite: False
==> Loading H36M img_25d... overwrite: False
==> Loading H36M img_2d... overwrite: False


### Verification with original dataloader

In [ ]:
train_loader_origin, test_loader_origin, _, _, datareader_origin = load_dataset(args)

Loading dataset...
H36M-GT


In [ ]:
train_data, test_data, train_labels, test_labels = datareader.get_sliced_data()
train_data_origin, test_data_origin, train_labels_origin, test_labels_origin = datareader_origin.get_sliced_data()

In [ ]:
# check data shape
assert train_data.shape == train_data_origin.shape, f'{train_data.shape} {train_data_origin.shape}'
assert test_data.shape == test_data_origin.shape, f'{test_data.shape} {test_data_origin.shape}'
assert train_labels.shape == train_labels_origin.shape, f'{train_labels.shape} {train_labels_origin.shape}'
assert test_labels.shape == test_labels_origin.shape, f'{test_labels.shape} {test_labels_origin.shape}'

In [ ]:
list(datareader.dt_dataset['train'].keys() - datareader.default_data_type_lsit)

['cam_3d', '2.5d_factor', 'joint_2d', 'joint3d_image', 'joints_2.5d_image']

In [ ]:
# check data values for each source
for source in tqdm(list(set(datareader_origin.dt_dataset['train']['source']))):
    loc1 = np.where(datareader_origin.dt_dataset['train']['source'] == source)[0]
    loc2 = np.where(datareader.dt_dataset['train']['source'] == source)[0]
    assert len(loc1) == len(loc2), f'{len(loc1)} {len(loc2)}'
    for key in list(datareader.dt_dataset['train'].keys() - datareader.default_data_type_lsit):
        data1 = datareader_origin.dt_dataset['train'][key][loc1].copy()
        data2 = datareader.dt_dataset['train'][key][loc2].copy()
        assert data1.shape == data2.shape, f'{source} {key} {data1.shape} {data2.shape}'
        if key == '2.5d_factor': atol = 3e-02
        elif key == 'joints_2.5d_image': atol = 20
        else: atol = 1e-07
        max_value = abs(data1 - data2).max()
        max_idx = abs(data1 - data2).argmax()
        if not np.allclose(data1, data2, atol=atol): print(f'{source} {key} {max_value}, {max_idx} {data1.flatten()[max_idx]} {data2.flatten()[max_idx]}') 
        if 'joint_2d' in key:
            cam_params1 = datareader_origin.dt_dataset['train']['cam_param'][loc1]
            cam_params2 = datareader.dt_dataset['train']['cam_param'][loc2]
            #W1, H1 = np.array([cam_param['W'] for cam_param in cam_params1]), np.array([cam_param['H'] for cam_param in cam_params1])
            W, H = np.array([cam_param['W'] for cam_param in cam_params2]), np.array([cam_param['H'] for cam_param in cam_params2])
            normalized_data1 = datareader.normalize(data1, W, H, '2d')
            normalized_data2 = datareader.normalize(data2, W, H, '2d')
            assert np.allclose(normalized_data1, normalized_data2, atol=1e-07), f'{source} {key} {abs(normalized_data1 - normalized_data2).max()}, {normalized_data1[abs(normalized_data1 - normalized_data2).argmax()]}, {noramlized_data2[abs(normalized_data1 - normalized_data2).argmax()]}'

100%|██████████| 600/600 [00:11<00:00, 50.43it/s]


In [ ]:
for item in glob('data/motion3d/MB3D_f243s81/*.yaml'):
    subset = os.path.basename(item).split('.')[0]
    if subset in blacklist: continue
    if subset not in dt_file_mapping.keys():
        print(subset)

In [ ]:
config_root = 'configs/pose3d/'
checkpoint_root = 'checkpoint/pose3d/'
blacklist = ['3DHP-GT-CAM_NO_FACTOR-POSEAUG_TEST_2929', 
             '3DHP-GT-CAM_NO_FACTOR-POSYNDA_TESTSET', 
             'H36M-CANONICALIZATION-GT-INPUT_FROM_3D_CANONICAL_SAME_Z-TR_S1_TS_S5678']

for item in glob('data/motion3d/MB3D_f243s81/*.yaml'):
    subset = os.path.basename(item).split('.')[0]
    dataset_name = subset.split('-')[0]
    if dataset_name != 'H36M': continue
    if 'NO_FACTOR' not in subset: continue
    if subset in blacklist: continue
    print(subset)
    datareader = DataReaderTotal(subset=subset, verbose=False)
    checkpoint = f'MB_train_{datareader.dataset_name}_{datareader.input_source.lower()}'
    if datareader.gt_mode == 'cam_3d': checkpoint += '_cam_no_factor'
    if datareader.canonical_type is not None: checkpoint += f'_input_from_canonical_3d_{datareader.canonical_type.lower()}'
    if 'S15678_TR_54138969_TS_OTHERS' in item: checkpoint += '_S15678_TR_54138969_TS_OTHERS'.lower()
    elif 'TR_S1_TS_S5678' in item: checkpoint += '_TR_S1_TS_S5678'.lower()
    elif 'TR_S03' in item: checkpoint += '_TR_S03'.lower()
    elif 'TS_S4710' in item: checkpoint += '_TS_S4710'.lower()
    print(checkpoint)
    #args, opts = get_opt_args_from_model_name(checkpoint)
    datareader_origin = DataReaderH36M(n_frames=243, 
                                       sample_stride=1, 
                                       data_stride_train=81, 
                                       data_stride_test=243, 
                                       dt_root = 'data/motion3d', 
                                       dt_file=dt_file_mapping[subset]+'.pkl', 
                                       input_mode=datareader.input_mode, 
                                       gt_mode=datareader.gt_mode)

    
    train_data, test_data, train_labels, test_labels = datareader.get_sliced_data()
    train_data_origin, test_data_origin, train_labels_origin, test_labels_origin = datareader_origin.get_sliced_data()
    
    print('check data shape')
    if train_data.shape != train_data_origin.shape: print(f'{train_data.shape} {train_data_origin.shape}')
    if test_data.shape != test_data_origin.shape: print(f'{test_data.shape} {test_data_origin.shape}')
    if train_labels.shape != train_labels_origin.shape: print(f'{train_labels.shape} {train_labels_origin.shape}')
    if test_labels.shape != test_labels_origin.shape: print(f'{test_labels.shape} {test_labels_origin.shape}')
    
    print('check data values for each source')
    for source in tqdm(list(set(datareader_origin.dt_dataset['train']['source']))):
        loc1 = np.where(datareader_origin.dt_dataset['train']['source'] == source)[0]
        loc2 = np.where(datareader.dt_dataset['train']['source'] == source)[0]
        for key in list(datareader.dt_dataset['train'].keys() - datareader.default_data_type_lsit):
            data1 = datareader_origin.dt_dataset['train'][key][loc1]
            data2 = datareader.dt_dataset['train'][key][loc2]
            #print(key, data1.shape, data2.shape)
            assert np.allclose(data1, data2, atol=1e-07), f'{source} {key} {abs(data1 - data2).max()}'
            if 'joint_2d' in key:
                cam_params1 = datareader_origin.dt_dataset['train']['cam_param'][loc1]
                cam_params2 = datareader.dt_dataset['train']['cam_param'][loc2]
                #W1, H1 = np.array([cam_param['W'] for cam_param in cam_params1]), np.array([cam_param['H'] for cam_param in cam_params1])
                W, H = np.array([cam_param['W'] for cam_param in cam_params2]), np.array([cam_param['H'] for cam_param in cam_params2])
                normalized_data1 = datareader.normalize(data1, W, H, '2d')
                normalized_data2 = datareader.normalize(data2, W, H, '2d')
                assert np.allclose(normalized_data1, normalized_data2, atol=1e-07), f'{source} {key} {abs(normalized_data1 - normalized_data2).max()}'

H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_FIXED_DIST_5-TR_S1_TS_S5678
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_fixed_dist_5_tr_s1_ts_s5678
check data shape
check data values for each source


100%|██████████| 120/120 [00:00<00:00, 255.27it/s]


H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_Z
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z
check data shape
check data values for each source


100%|██████████| 600/600 [00:09<00:00, 62.09it/s]


H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-S15678_TR_54138969_TS_OTHERS
MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_dist_s15678_tr_54138969_ts_others


KeyError: 'cam_3d'

In [ ]:
datareader.train_subject, datareader.test_subject, datareader.data_type_list

(['S1', 'S5', 'S6', 'S7', 'S8'],
 ['S1', 'S5', 'S6', 'S7', 'S8'],
 ['cam_3d',
  'joint_2d_from_canonical_3d',
  'cam_3d_from_canonical_3d',
  'source',
  'cam_param',
  'camera_name',
  'action',
  'confidence'])

In [ ]:
datareader.dt_dataset['train']['source']

array([], dtype=float64)

In [ ]:
subset

'H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-S15678_TR_54138969_TS_OTHERS'

In [ ]:
datareader = DataReaderTotal(subset='H36M-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_SAME_DIST-S15678_TR_54138969_TS_OTHERS',
                             verbose=False)

{'cam_list': [], 'canonical_type': 'same_dist', 'data_type_list': ['cam_3d', 'joint_2d_from_canonical_3d', 'cam_3d_from_canonical_3d'], 'dataset_name': 'h36m', 'gt_mode': 'cam_3d', 'input_mode': 'joint_2d_from_canonical_3d', 'input_source': 'GT', 'test_cam': ['60457274', '55011271', '58860488'], 'test_subject': ['S1', 'S5', 'S6', 'S7', 'S8'], 'train_cam': ['54138969'], 'train_subject': ['S1', 'S5', 'S6', 'S7', 'S8']}


In [ ]:
datareader.yaml_data

{'cam_list': [],
 'canonical_type': 'same_dist',
 'data_type_list': ['cam_3d',
  'joint_2d_from_canonical_3d',
  'cam_3d_from_canonical_3d'],
 'dataset_name': 'h36m',
 'gt_mode': 'cam_3d',
 'input_mode': 'joint_2d_from_canonical_3d',
 'input_source': 'GT',
 'test_cam': ['60457274', '55011271', '58860488'],
 'test_subject': ['S1', 'S5', 'S6', 'S7', 'S8'],
 'train_cam': ['54138969'],
 'train_subject': ['S1', 'S5', 'S6', 'S7', 'S8']}

In [ ]:
datareader_origin.dt_dataset['train'].keys()

dict_keys(['joint_2d_from_canonical_3d', 'confidence', 'joint3d_image_from_canonical_3d', 'joints_2.5d_image_from_canonical_3d', '2.5d_factor_from_canonical_3d', 'camera_name', 'action', 'source', 'frame', 'world_3d', 'cam_3d_from_canonical_3d', 'cam_param', 'world_3from_canonical_3d'])

### Multiple subset - DataReaderTotalGroup

In [2]:
from lib.data.datareader_total import DataReaderTotalGroup, DataReaderTotal
subset_list = ['H36M-GT-CAM_NO_FACTOR-STEP_ROT_1.5-TR_S1_TS_S5678', 'H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678']
datareader = DataReaderTotalGroup(subset_list=subset_list)
checkpoint = 'MB_train_h36m_gt_cam_no_factor_steprot_1.5+original_tr_s1_ts_s5678'
args, opts = get_opt_args_from_model_name(checkpoint, mode='latest')

H36M-GT-CAM_NO_FACTOR-STEP_ROT_1.5-TR_S1_TS_S5678
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d_steprot_1.5...
[overwrite: False] ==> Loading H36M img_2d_steprot_1.5...
H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d_steprot_0.5...
[overwrite: False] ==> Loading H36M img_2d_steprot_0.5...
H36M-GT-CAM_NO_FACTOR-TR_S1_TS_S5678
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d...
[overwrite: False] ==> Loading H36M img_2d...
configs/pose3d/MB_train_h36m_gt_cam_no_factor_steprot_1.5+original_tr_s1_ts_s5678.yaml
True


In [3]:
train_data, test_data, train_labels, test_labels = datareader.get_sliced_data()

In [4]:
train_data.shape, test_data.shape, train_labels.shape, test_labels.shape

((9148, 243, 17, 3),
 (10312, 243, 17, 3),
 (9148, 243, 17, 3),
 (10312, 243, 17, 3))

In [5]:
num_test_frames, action_clips, factor_clips, source_clips, frame_clips, gt_clips, actions = datareader.get_clip_info(args, len(test_labels))

In [6]:
action_clips.shape, factor_clips.shape, source_clips.shape, frame_clips.shape, gt_clips.shape, actions.shape

((20624, 243),
 (20624, 243),
 (20624, 243),
 (20624, 243),
 (20624, 243, 17, 3),
 (5245504,))

In [7]:
len(frame_clips)

10312

In [8]:
10312/2

5156.0

In [9]:
frame_clips[5156], frame_clips[-1]

(array([1311376, 1311377, 1311378, 1311379, 1311380, 1311381, 1311382,
        1311383, 1311384, 1311385, 1311386, 1311387, 1311388, 1311389,
        1311390, 1311391, 1311392, 1311393, 1311394, 1311395, 1311396,
        1311397, 1311398, 1311399, 1311400, 1311401, 1311402, 1311403,
        1311404, 1311405, 1311406, 1311407, 1311408, 1311409, 1311410,
        1311411, 1311412, 1311413, 1311414, 1311415, 1311416, 1311417,
        1311418, 1311419, 1311420, 1311421, 1311422, 1311423, 1311424,
        1311425, 1311426, 1311427, 1311428, 1311429, 1311430, 1311431,
        1311432, 1311433, 1311434, 1311435, 1311436, 1311437, 1311438,
        1311439, 1311440, 1311441, 1311442, 1311443, 1311444, 1311445,
        1311446, 1311447, 1311448, 1311449, 1311450, 1311451, 1311452,
        1311453, 1311454, 1311455, 1311456, 1311457, 1311458, 1311459,
        1311460, 1311461, 1311462, 1311463, 1311464, 1311465, 1311466,
        1311467, 1311468, 1311469, 1311470, 1311471, 1311472, 1311473,
      